In [7]:
%matplotlib inline

# This nb is modified from DGL documentation
- https://docs.dgl.ai/en/0.8.x/tutorials/blitz/1_introduction.html 
- https://docs.dgl.ai/en/0.8.x/tutorials/blitz/3_message_passing.html \
With some additional material...


How Does DGL Represent A Graph?
===============================

By the end of this tutorial you will be able to:

-  Construct a graph in DGL from scratch.
-  Assign node and edge features to a graph.
-  Query properties of a DGL graph such as node degrees and
   connectivity.
-  Transform a DGL graph into another graph.
-  Load and save DGL graphs.

(Time estimate: 16 minutes)


DGL Graph Construction
----------------------

DGL represents a directed graph as a ``DGLGraph`` object. You can
construct a graph by specifying the number of nodes in the graph as well
as the list of source and destination nodes.  Nodes in the graph have
consecutive IDs starting from 0.

For instance, the following code constructs a directed star graph with 5
leaves. The center node's ID is 0. The edges go from the
center node to the leaves.




In [12]:
import dgl
import numpy as np
import torch

g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
# Equivalently, PyTorch LongTensors also work.
g = dgl.graph((torch.LongTensor([0, 0, 0, 0, 0]), torch.LongTensor([1, 2, 3, 4, 5])), num_nodes=6)

# You can omit the number of nodes argument if you can tell the number of nodes from the edge list alone.
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))

ModuleNotFoundError: No module named 'dgl'

Edges in the graph have consecutive IDs starting from 0, and are
in the same order as the list of source and destination nodes during
creation.




In [13]:
# Print the source and destination nodes of every edge.
print(g.edges())

NameError: name 'g' is not defined

<div class="alert alert-info"><h4>Note</h4><p>``DGLGraph``'s are always directed to best fit the computation
   pattern of graph neural networks, where the messages sent
   from one node to the other are often different between both
   directions. If you want to handle undirected graphs, you may consider
   treating it as a bidirectional graph. See `Graph
   Transformations`_ for an example of making
   a bidirectional graph.</p></div>




### We can add and remove edges

In [ ]:
g1 = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))
g1 = dgl.add_edges(g1, [1, 2, 2], [0, 0, 5])
print(g1.edges())

g2 = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))
g2 = dgl.remove_edges(g2,[0,1])
print(g2.edges())

### We can visualize the graph using `networkx`

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, g_ in enumerate([g, g1, g2]):
    nx.draw(g_.to_networkx(), with_labels=True, ax=axes[i])

### We can also convert a graph to a bidirectional one

In [10]:
g = dgl.to_bidirected(g)

NameError: name 'dgl' is not defined

Assigning Node and Edge Features to Graph
-----------------------------------------

Many graph data contain attributes on nodes and edges.
Although the types of node and edge attributes can be arbitrary in real
world, ``DGLGraph`` only accepts attributes stored in tensors (with
numerical contents). Consequently, an attribute of all the nodes or
edges must have the same shape. In the context of deep learning, those
attributes are often called *features*.

You can assign and retrieve node and edge features via ``ndata`` and
``edata`` interface.




In [ ]:
# Assign a 3-dimensional node feature vector for each node.
g.ndata['x'] = torch.randn(6, 3)
# Assign a 4-dimensional edge feature vector for each edge.
g.edata['a'] = torch.randn(g.num_edges(), 4)
# Assign a 5x4 node feature matrix for each node.  Node and edge features in DGL can be multi-dimensional.
g.ndata['y'] = torch.randn(6, 5, 4)

print(g.edata['a'])

<div class="alert alert-info"><h4>Note</h4><p>The vast development of deep learning has provided us many
   ways to encode various types of attributes into numerical features.
   Here are some general suggestions:

   -  For categorical attributes (e.g. gender, occupation), consider
      converting them to integers or one-hot encoding.
   -  For variable length string contents (e.g. news article, quote),
      consider applying a language model.
   -  For images, consider applying a vision model such as CNNs.

   You can find plenty of materials on how to encode such attributes
   into a tensor in the `PyTorch Deep Learning
   Tutorials <https://pytorch.org/tutorials/>`__.</p></div>




Querying Graph Structures
-------------------------

``DGLGraph`` object provides various methods to query a graph structure.




In [ ]:
print(g.num_nodes())
print(g.num_edges())
# Out degrees of the center node
print(g.out_degrees(0))
# In degrees of the center node - note that the graph is directed so the in degree should be 0.
print(g.in_degrees(0))

# Message passing

<img src="./messages.jpeg"  width="500" height="665">

### In DGL the `update_all` function can be used to perform all three steps in one go

### First load some ones onto all the nodes

In [11]:
import dgl.function as fn

ones = torch.ones(4)
g.ndata['count'] = dgl.broadcast_nodes(g, ones)
print(g.ndata['count'])

ModuleNotFoundError: No module named 'dgl'

### Then for each node, let's add up its neighbors

In [ ]:
g.update_all(
    fn.copy_u('count', 'm'), # <-- propagate
    fn.sum('m', 'sum')       # <-- aggregate
    )

### Let's look at the resulting feature

In [ ]:
print(g.ndata['sum'])

### We can define our own custom aggregate function

In [ ]:
def my_function(nodes):
    mail = nodes.mailbox['m']
    my_sum = torch.sum(mail,dim=1) / g.num_nodes()
    return {'my sum': my_sum}

In [ ]:
g.update_all(
    fn.copy_u('count', 'm'), # <-- propagate
    my_function              # <-- aggregate
    )

In [ ]:
print(g.ndata['my sum'])

### Similarly, we can create our own propagation function

In [ ]:
def propagate(edges):

    return {'m': edges.src['count'], 'a': edges.data['a']}

In [ ]:
def sum_count_times_a(nodes):
    mail      = nodes.mailbox['m']
    edge_mail = nodes.mailbox['a']

    #edge-weighted sum
    my_sum    = torch.sum(mail * edge_mail, dim=1)

    return {'my sum': my_sum}

In [ ]:
g.update_all(
    propagate,         # <-- propagate
    sum_count_times_a  # <-- aggregate
    )

In [ ]:
print(g.ndata['my sum'])